<a href="https://colab.research.google.com/github/bilal-rachik/detection-in-contract-management/blob/master/detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwriting,signatures and initials detection in contract management __Part 3 __ Detectron2.
Dans l'article précédent, nous avons formé notre modèle de détection des écritures manuscrites, signatures , intials  à l'aide de l'API de détection d'objets de TensorFlow.

Dnas cet article nous allons découvrire un autre framework  pour la construction des modèles de détection d'objets et de segmentation d'images, qui s'appelle Detectron2.


Dans cet article, nous allons découvrir un autre cadre pour la construction de modèles de détection d'objets et de segmentation d'images, qui s'appelle Detectron2.

Parallèlement à la sortie de la version 1.3 de PyTorch, Facebook a également publié une réécriture fondamentale de son cadre de détection d'objets Detectron . Le nouveau framework s'appelle Detectron2 et est maintenant implémenté dans PyTorch .

Detectron2 nous permet facilement de construire des modèles de détection d'objets. Cet article vous aidera à démarrer avec Detectron2 en apprenant à  former votre propre modèle.


# Installer Detectron2
L'installation de Detectron2 est facile par rapport à d'autres frameworks de détection d'objets comme l'API de détection d'objets Tensorflow.

# Installer Detectron2
L'installation de Detectron2 est facile par rapport à d'autres frameworks de détection d'objets comme l'API de détection d'objets Tensorflow.

# Installation sur Google Colab
Si vous travaillez dans Google Colab, il peut être installé avec les quatre lignes suivantes:

In [0]:

# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

In [0]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

# Installation sur une machine locale
Si vous travaillez sur une machine locale, ce n'est pas aussi simple que cela, mais toujours gérable.

je vous invite à  consulter le guide [d'installation officiel](https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md#common-installation-issues) 



Annoter les images
C'est déja fait et bien expliqué  dans mes [article precedent](https://github.com/bilal-rachik/detection-in-contract-management/blob/master/Handwriting%2Csignatures%20and%20initials%20%20detection%20in%20contract%20management%20%20part%201.ipynb) . 

Nous avons utilisé localement le logiciel open source VOTT développé par Microsoft, pour annoter les contrats.

Notre ensemble de données est composé d'une variété de contrats du gouvernement télécharger depuis ce [lien]( https://www.gsa.gov/real-estate/real-estate-services/leasing-policy-procedures/lease-documents)  .

L'utilisation de VOTT nous a permis de produire un ensemble de formation de 300  images étiquetées à partir d'un échantillon de contrats du gouvernement en quelques heures. Nous avons tiré notre ensemble de tests de 90 images contractuelles supplémentaires.

Une fois que vous avez fini d'annoter votre jeu de données d'image, exportez vos actifs dans VOTT au format Pascal VOC.
Structure d'ensemble de données attendue pour Pascal VOC est la suivante :


In [0]:
pascalVOC-export/
  Annotations/
  ImageSets/
    Main/
      train.txt
      test.txt
      # train.txt or val.txt, if you use these splits
  JPEGImages/

* annotations: Le dossier contient des fichiers XML, un fichier par image. Il stocke des métadonnées sur une image comme un dossier où l'image est stockée, son nom de fichier, sa taille et chaque boîte englobante...etc
* Ensuite, nous avons un JPEGImages dossier, il contient tous nos images jpg.
* train.txt et test.txt : est notre ensemble de données divisé pour former et tester le modèle.

# Enregistrer le jeu de données

Pour que Detectron2 sache comment obtenir l'ensemble de données, nous devons l'enregistrer et éventuellement enregistrer des métadonnées pour votre ensemble de données.
Le processus est bien décrit avec des détails dans la [documentation Detectron2](https://detectron2.readthedocs.io/tutorials/datasets.html) .

En général, Detectron2 utilise son propre format pour la représentation des données qui est similaire aux annotations JSON de COCO. Il s'agit d'implémenter une fonction qui renvoie toutes les informations nécessaires sur les données sous forme de liste et en transmettant le résultat à DatasetCatalog.register



In [0]:
def get_dicts():
  ...
  return list[dict] in the following format

from detectron2.data import DatasetCatalog
DatasetCatalog.register("my_dataset", get_dicts)

Pour l'ensemble de données qui est déjà au format COCO, Detectron2 fournit la `register_coco_instances` fonction qui `load_coco_json` vous enregistrera et ajoutera des métadonnées sur votre ensemble de données.
Les métadonnées sont un mappage de valeurs-clés qui fournit des informations sur l'ensemble de données comme les noms des classes, les couleurs des classes, la racine des fichiers, etc. qui sont accessibles via `MetadataCatalog.get(dataset_name).some_metadata.`

qui pourrait être une inspiration pour nous.


Dans notre cas, nous avons l'ensemble de données au format Pascal VOC et il n'y a pas de chargeur à usage général pour ce format. Heureusement, Detectron2 a une implémentation pour l'enregistrement des jeux de données Pascal VOC (voir [detectron2/detectron2/data/datasets/pascal_voc.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/data/datasets/pascal_voc.py) 

In [0]:
import numpy as np
import os
import xml.etree.ElementTree as ET
from fvcore.common.file_io import PathManager
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode

# fmt: off
CLASS_NAMES= ['signature','Handwrit','intial']
# fmt: on

def load_voc_instances(dirname: str, split: str):
    """
    Load Pascal VOC detection annotations to Detectron2 format.
    Args:
        dirname: Contain "Annotations", "ImageSets", "JPEGImages"
        split (str): one of "train", "test", "val", "trainval"
    """
    with PathManager.open(os.path.join(dirname, "ImageSets", "Main", split + ".txt")) as f:
        fileids = np.loadtxt(f, dtype=np.str)

    # Needs to read many small annotation files. Makes sense at local
    annotation_dirname = PathManager.get_local_path(os.path.join(dirname, "Annotations/"))
    dicts = []
    for fileid in fileids:
        anno_file = os.path.join(annotation_dirname, fileid + ".xml")
        jpeg_file = os.path.join(dirname, "JPEGImages", fileid + ".jpg")

        with PathManager.open(anno_file) as f:
            tree = ET.parse(f)

        r = {
            "file_name": jpeg_file,
            "image_id": fileid,
            "height": int(tree.findall("./size/height")[0].text),
            "width": int(tree.findall("./size/width")[0].text),
        }
        instances = []

        for obj in tree.findall("object"):
            cls = obj.find("name").text
            # We include "difficult" samples in training.
            # Based on limited experiments, they don't hurt accuracy.
            # difficult = int(obj.find("difficult").text)
            # if difficult == 1:
            # continue
            bbox = obj.find("bndbox")
            bbox = [float(bbox.find(x).text) for x in ["xmin", "ymin", "xmax", "ymax"]]
            # Original annotations are integers in the range [1, W or H]
            # Assuming they mean 1-based pixel indices (inclusive),
            # a box with annotation (xmin=1, xmax=W) covers the whole image.
            # In coordinate space this is represented by (xmin=0, xmax=W)
            bbox[0] -= 1.0
            bbox[1] -= 1.0
            instances.append(
                {"category_id": CLASS_NAMES.index(cls), "bbox": bbox, "bbox_mode": BoxMode.XYXY_ABS}
            )
        r["annotations"] = instances
        dicts.append(r)
    return dicts

In [0]:
dataset_name='sig-hand_int_'
dirname ='/content/drive/My Drive/my-project/detectron/PascalVOC-export' 

for split in ["train", "val"]:
  DatasetCatalog.register(dataset_name+split,lambda: load_voc_instances(dirname, split))
  MetadataCatalog.get(dataset_name+split).set(thing_classes=CLASS_NAMES,dirname=dirname,split=split)

Vous pouvez basculer vers le jeu de données de test avec option --split test

Pour vérifier que le chargement des données est correct, visualisons les annotations d'échantillons sélectionnés au hasard dans l'ensemble d'apprentissage:


In [0]:
from google.colab.patches import cv2_imshow
import random
import cv2
from detectron2.utils.visualizer import Visualizer
samples=5
split ='train'

dataset_dicts = DatasetCatalog.get(dataset_name+split)
for d in random.sample(dataset_dicts, samples):
        img = cv2.imread(d["file_name"])
        visualizer = Visualizer(img[:, :, ::-1],
                                metadata=MetadataCatalog.get(dataset_name+split),
                                scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        cv2_imshow(vis.get_image()[:, :, ::-1])

Nous sommes prêts à former notre modèle.

# Former un modèle personnalisé


In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ('sig-hand_int_train',)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [0]:
cfg.OUTPUT_DIR

In [0]:
%cd output/

In [0]:
ls